In [1]:
import numpy as np
import pandas as pd

In [2]:
def Info_Ent(Ydata: pd.Series) -> float:
    '''
    计算当前样本集的信息熵
    Ydata: 样本的标签
    '''
    n_sample = len(Ydata) # 样本数
    prob = Ydata.value_counts() / n_sample # 样本集中各类标签出现的概率
    info_ent = -(prob * np.log2(prob)).sum() # 信息熵
    return info_ent

In [5]:
def Info_Gain(Xdata: pd.DataFrame, Ydata: pd.Series, attr) -> float:
    '''
    计算当前样本集在属性attr下的信息增益
    Xdata: 样本的输入特征
    Ydata: 样本的标签
    attr: 属性
    '''
    info_ent = Info_Ent(Ydata) # 样本集的信息熵
    vals = Xdata[attr].unique() # 属性attr的所有取值
    n_sample = len(Xdata) # 样本数
    attr_ent = 0 # 样本集在属性attr下的信息熵
    for v in vals: # 取出一个属性值
        idx = (Xdata[attr] == v) # 筛选出样本集中属性attr上取值为v的所有样本
        attr_ent += idx.sum() / n_sample * Info_Ent(Ydata[idx]) # 累加
    info_gain = info_ent - attr_ent # 信息增益
    return info_gain

In [25]:
class DecisionTree:

    def __init__(self):
        pass

    def fit(self, Xtrain: pd.DataFrame, Ytrain: pd.Series) -> dict:
        '''
        训练决策树
        Xtrain: 训练集的输入特征
        Ytrain: 训练集的标签
        '''
        attr_set = Xtrain.columns.values  # 属性集
        self.all_attr_vals = {} # 存放各属性所有可能的取值
        # 获取各属性所有可能的取值
        for attrn in Xtrain.columns:
            self.all_attr_vals[attrn] = Xtrain[attrn].unique()
        self.tree = self.createTree(Xtrain, Ytrain, attr_set)  # 由训练样本训练决策树
        return self.tree

    def createTree(self, Xdata: pd.DataFrame, Ydata: pd.Series, attr_set: np.ndarray):
        '''
        以递归方式创建决策树
        Xdata: 样本的输入特征
        Ydata: 样本的标签
        attr_set: 当前属性集
        '''
        # 若当前样本集为空，则直接将其标为空节点，后续还需要将该空节点根据父节点的样本集标为对应的叶节点（即具体的标签）
        if len(Ydata) == 0:
            return {}
        # 若当前样本集中所有的样本的标签均属于同一类，则直接将其标为叶节点
        if len(Ydata.unique()) == 1:
            return Ydata.iloc[0]
        # 若当前属性集为空或者所有样本在所有属性上的取值都一样，则直接将其标为叶节点，并且标签类别设为当前样本集中标签最多的类别
        if len(attr_set) == 0 or len(Xdata.drop_duplicates()) == 1:
            return Ydata.value_counts().index[0]
        tree = {}
        gains = np.array([])  # 用于记录不同属性下对应的信息增益
        for attr in attr_set:  # 从当前属性集从取出一个属性attr
            gains = np.append(gains, Info_Gain(Xdata, Ydata, attr))  # 计算attr下对应的信息增益并加入gains中
        attr_idx = gains.argmax()  # gains中最大信息增益值的索引
        div_attr = attr_set[attr_idx]  # 选定划分属性
        childs = {}  # 存放子节点
        tree[div_attr] = childs
        new_attr_set = np.delete(attr_set, attr_idx)  # 将划分属性从属性集中删去，为子节点生成新的属性集
        attr_vals = self.all_attr_vals[div_attr]  # 划分属性的所有取值
        for val in attr_vals:  # 从划分属性中取出一个值val
            samp_idx = (Xdata[div_attr] == val)  # 筛选出当前样本集中在划分属性上取值为val的所有样本
            X = Xdata[samp_idx]
            Y = Ydata[samp_idx]
            child = self.createTree(X, Y, new_attr_set)  # 以递归方式生成子树作为当前节点的子节点
            childs[val] = child
            # 将上述所说的空节点标为叶子节点：用当前节点中标签类别最多的样本对该空节点进行标记
            if childs[val] == {}:
                childs[val] = Ydata.value_counts().index[0]
        return tree

    def predict_1(self, x: pd.Series):
        '''
        预测一个样本的类别
        x: 样本的特征输入
        '''
        cur = self.tree  # 游标，用于实现从树的根节点到叶节点的遍历
        while type(cur) == dict:  # 若遍历到的是非叶子节点就继续往下遍历
            attr = list(cur.keys())[0]  # 当前节点的划分属性
            cur = cur[attr]
            cur = cur[x[attr]]
        return cur

    def predict_n(self, Xtest: pd.DataFrame):
        '''
        预测一批样本的类别
        Xtest: 测试样本的特征输入
        '''
        Ypred = []  # 记录样本的预测结果
        for idx in Xtest.index:
            Ypred.append(self.predict_1(Xtest.loc[idx]))
        return Ypred

    def evaluate(self, Xtest, Ytest):
        '''
        计算模型在测试集合的准确度
        Xtest: 测试样本的特征输入
        Ytest: 测试样本的真实标签
        '''
        Ypred = self.predict_n(Xtest)
        accurancy = (Ypred == Ytest).sum() / len(Ytest)
        return accurancy

In [26]:
# 测试
df = pd.read_csv('西瓜2.0.csv', index_col='编号')
Xtrain = df.drop('好瓜', axis=1)
Ytrain = df['好瓜']
classifier = DecisionTree()
print(classifier.fit(Xtrain, Ytrain))

{'纹理': {'清晰': {'根蒂': {'蜷缩': '是', '稍蜷': {'色泽': {'青绿': '是', '乌黑': {'触感': {'硬滑': '是', '软粘': '否'}}, '浅白': '是'}}, '硬挺': '否'}}, '稍糊': {'触感': {'硬滑': '否', '软粘': '是'}}, '模糊': '否'}}


In [27]:
df1 = pd.read_csv('西瓜（测试）.csv', index_col='编号')
Xtest = df1.drop('好瓜', axis=1)
Ytest = df1['好瓜']
print(classifier.predict_n(Xtest))

['是', '是', '否', '否']


In [29]:
print(f'精度为：{100 * classifier.evaluate(Xtest, Ytest)}%')

精度为：100.0%
